In [1]:
#[Hipótese 1] Data por CNAE

In [1]:
# Maior movimentação, elencar o maior grupo de tipos de empresa.
# 1. Criar faixas de horário de 3 em 3 horas com transações efetuadas nesse horário - por empresa.
# 2. Plotar histograma de faixa de horário de transação. 1 gráfico por CNAE (não empresa).
# 3. Verificar a possibilidade de clusterização utilizando KNN caso gráfico de um CNAE seja muito divergente 
#    (sem faixa de horários muito contrastantes), verificando se cabem clusters de clientes, dado horário de transações.

# Faixas de horário:
# 02 h as 05 h
# 05 h as 08 h
# 08 h as 11 h
# 11 h as 14 h
# 14 h as 17 h
# 17 h as 20 h
# 20 h as 23 h
# 23 h as 02 h

In [2]:
# importar arquivo como csv

import pandas as pd

combined_csv = pd.read_csv("../combined_csv.csv", skiprows = 1, names =["dominio","empresa","3","datahora","5","6","7","8"])

print(combined_csv.shape)

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1499999, 8)


In [3]:
# removendo colunas desnecessarias

combined_csv.drop("3", inplace=True, axis=1)
combined_csv.drop("5", inplace=True, axis=1)
combined_csv.drop("6", inplace=True, axis=1)
combined_csv.drop("7", inplace=True, axis=1)
combined_csv.drop("8", inplace=True, axis=1)

In [4]:
# tratamento de data e hora

from datetime import datetime

data_e_hora_parsed = pd.to_datetime(combined_csv.loc[:, "datahora"], format="%Y-%m-%dT%H:%M:%S.000Z",errors="coerce")

combined_csv["datahora"] = data_e_hora_parsed

print(combined_csv.head(4))

                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [5]:
# limpeza dos campos NaT

df_datas_corretas = combined_csv[combined_csv["datahora"].notnull()] 
df_datas_incorretas = combined_csv[combined_csv["datahora"].isnull()]

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa            datahora  
0  21404ebaa0106ca20ae283c89271 2018-11-19 16:40:45  
1  510766392a95afd41c6bd526381a 2018-12-07 09:56:37  
2  21404ebaa0106ca20ae283c89271 2018-10-18 08:29:47  
3  8ef4d40b852168ea8e3bbc4c89a0 2018-10-15 09:18:55  


In [6]:
# transformando em horário

df_horarios_transacoes = df_datas_corretas["datahora"].dt.hour

df_datas_corretas["hora"] = df_horarios_transacoes

df_datas_corretas.drop("datahora", inplace=True, axis=1)

print(df_datas_corretas.shape)
print(df_datas_corretas.head(4))

(1499980, 3)
                                             dominio  \
0  6139646137356433363237396637323937336161383832...   
1  6139646137356433363237396637323937336161383832...   
2  6139646137356433363237396637323937336161383832...   
3  6139646137356433363237396637323937336161383832...   

                        empresa  hora  
0  21404ebaa0106ca20ae283c89271    16  
1  510766392a95afd41c6bd526381a     9  
2  21404ebaa0106ca20ae283c89271     8  
3  8ef4d40b852168ea8e3bbc4c89a0     9  


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
D:\Anaconda\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [19]:
# Agrupando as empresas e criando um vetor de horário por cada transação

empresas = df_datas_corretas.groupby(["empresa"])

print(empresas.groups)

{'002d36b2edaa62d8f4ec0cae92bf': Int64Index([  10602,   10689,   10777,   10797,   19299,   19315,   19631,
              19763,   40659,   40752,
            ...
            1480678, 1480760, 1481702, 1486989, 1487063, 1487139, 1488211,
            1489643, 1496549, 1496885],
           dtype='int64', length=320), '0049a955027a5ec4558cd0797637': Int64Index([   2236,    2629,    3910,    3925,    4444,   10673,   11001,
              12526,   19498,   20075,
            ...
            1476770, 1477119, 1481138, 1481150, 1482287, 1482313, 1488066,
            1496279, 1498086, 1498174],
           dtype='int64', length=773), '005180a2c46d488615304414075f': Int64Index([   3499,    3599,    4278,    4468,   12419,   15983,   16930,
              21537,   26394,   26597,
            ...
            1474195, 1474244, 1474406, 1475418, 1475728, 1475941, 1488074,
            1488208, 1497505, 1497569],
           dtype='int64', length=697), '00852b4319fd3b837a853e3942a1': Int64Index([   6232

TypeError: 'Pandas' object does not support item assignment

KeyboardInterrupt: 